# Tool Call Success Evaluator

## Objective
This sample demonstrates how to use tool call success evaluator on agent data. The supported input formats include:
- simple data such as strings and `dict` describing agent responses;
- user-agent conversations in the form of list of agent messages. 

## Time

You should expect to spend about 20 minutes running this notebook. 

## Before you begin
For quality evaluation, you need to deploy a `gpt` model supporting JSON mode. We recommend a model `gpt-4o` or `gpt-4o-mini` for their strong reasoning capabilities.    

### Prerequisite
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **AZURE_AI_PROJECT** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the model for this AI-assisted evaluator, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.


The Tool Call Success evaluator determines whether tool calls done by an AI agent includes failures or not.

This evaluator focuses solely on tool call results and tool definitions, disregarding user's query to the agent, conversation history and agent's final response. Although tool definitions is optional, providing them can help the evaluator better understand the context of the tool calls made by the agent. Please note that this evaluator validates tool calls for potential technical failures like errors, exceptions, timeouts and empty results (only in cases where empty results could indicate a failure). It does not assess the correctness or the tool result itself, like mathematical errors and unrealistic field values like name="668656".

Scoring is binary:
- TRUE: All tool calls were successful
- FALSE: At least one tool call failed

This evaluation focuses on measuring the technical success of tool execution, not the semantic correctness of the results.

Tool Call Success requires following input:
- Response - Response from Agent (or any GenAI App). This can be a single text response or a list of messages generated as part of Agent Response. The evaluator examines tool call results within the response.
- Tool Definitions - (Optional) Tool(s) definition used by Agent. Providing tool definitions helps the evaluator better understand the context of the tool calls made by the agent.


### Initialize Tool Call Success Evaluator


In [2]:
import os
from azure.ai.evaluation._evaluators._tool_success import _ToolSuccessEvaluator
from azure.ai.evaluation import AzureOpenAIModelConfiguration
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)


tool_call_success = _ToolSuccessEvaluator(model_config=model_config)

Class _ToolSuccessEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Samples

#### Evaluating Successful Tool Calls

In [3]:
# Successful tool execution
successful_response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_123",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_123",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "15°C", "condition": "partly cloudy", "humidity": "68%", "wind": "8 mph NW"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The current weather in Seattle is partly cloudy with a temperature of 15°C.",
            }
        ],
    }
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

result = tool_call_success(response=successful_response, tool_definitions=tool_definitions)
pprint(result)

{'tool_success': 1.0,
 'tool_success_completion_tokens': 60,
 'tool_success_finish_reason': 'stop',
 'tool_success_model': 'gpt-4.1-2025-04-14',
 'tool_success_prompt_tokens': 2407,
 'tool_success_reason': 'The fetch_weather tool returned a result containing '
                        'temperature, condition, humidity, and wind, with no '
                        'indication of error or technical failure. '
                        "{'failed_tools': '', 'success': True}",
 'tool_success_result': 'pass',
 'tool_success_sample_input': '[{"role": "user", "content": '
                              '"{\\"tool_calls\\": \\"[TOOL_CALL] '
                              'fetch_weather(location=\\\\\\"Seattle\\\\\\")\\\\n[TOOL_RESULT] '
                              "{'temperature': '15\\\\u00b0C', 'condition': "
                              "'partly cloudy', 'humidity': '68%', 'wind': '8 "
                              'mph NW\'}\\", \\"tool_definitions\\": '
                              '\\"TOOL

#### Tool Definition as Single Dict

#### Response as String (str)

In [4]:
# Response as a simple string containing tool call information
# This format is less common but still valid for the evaluator
response_str = """Tool call executed: fetch_weather(location="Paris")
Tool result: {"temperature": "18°C", "condition": "clear sky", "humidity": "55%"}
The weather in Paris is currently clear with a temperature of 18°C."""

tool_definition = {
    "name": "fetch_weather",
    "description": "Fetches the weather information for the specified location.",
    "parameters": {
        "type": "object",
        "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
    },
}

result = tool_call_success(response=response_str, tool_definitions=tool_definition)
pprint(result)

Agent response could not be parsed, falling back to original response: Tool call executed: fetch_weather(location="Paris")
Tool result: {"temperature": "18°C", "condition": "clear sky", "humidity": "55%"}
The weather in Paris is currently clear with a temperature of 18°C.
Failed to filter tool definitions, returning original list. Error: 'str' object has no attribute 'get'
Tool definitions could not be parsed, falling back to original definitions: {'name': 'fetch_weather', 'description': 'Fetches the weather information for the specified location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to fetch weather for.'}}}}


{'tool_success': 1.0,
 'tool_success_completion_tokens': 67,
 'tool_success_finish_reason': 'stop',
 'tool_success_model': 'gpt-4.1-2025-04-14',
 'tool_success_prompt_tokens': 2441,
 'tool_success_reason': 'The tool fetch_weather returned a result with '
                        'temperature, condition, and humidity fields '
                        'populated. There are no indications of technical '
                        'errors, exceptions, or empty/invalid responses. '
                        "{'failed_tools': '', 'success': True}",
 'tool_success_result': 'pass',
 'tool_success_sample_input': '[{"role": "user", "content": '
                              '"{\\"tool_calls\\": \\"Tool call executed: '
                              'fetch_weather(location=\\\\\\"Paris\\\\\\")\\\\nTool '
                              'result: {\\\\\\"temperature\\\\\\": '
                              '\\\\\\"18\\\\u00b0C\\\\\\", '
                              '\\\\\\"condition\\\\\\": \\\\\\"clear '
 

In [5]:
# Successful tool execution with single tool definition dict
successful_response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_nyc_weather",
                "name": "fetch_weather",
                "arguments": {"location": "New York"},
            }
        ],
    },
    {
        "tool_call_id": "call_nyc_weather",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "22°C", "condition": "sunny", "humidity": "45%"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "The weather in New York is currently sunny with a temperature of 22°C.",
            }
        ],
    }
]

tool_definition_dict = {
    "name": "fetch_weather",
    "description": "Fetches the weather information for the specified location.",
    "parameters": {
        "type": "object",
        "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
    },
}

result = tool_call_success(response=successful_response, tool_definitions=tool_definition_dict)
pprint(result)

Failed to filter tool definitions, returning original list. Error: 'str' object has no attribute 'get'
Tool definitions could not be parsed, falling back to original definitions: {'name': 'fetch_weather', 'description': 'Fetches the weather information for the specified location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to fetch weather for.'}}}}
Tool definitions could not be parsed, falling back to original definitions: {'name': 'fetch_weather', 'description': 'Fetches the weather information for the specified location.', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to fetch weather for.'}}}}


{'tool_success': 0.0,
 'tool_success_completion_tokens': 66,
 'tool_success_finish_reason': 'stop',
 'tool_success_model': 'gpt-4.1-2025-04-14',
 'tool_success_prompt_tokens': 2430,
 'tool_success_reason': 'The fetch_weather tool returned a result with '
                        'temperature, condition, and humidity fields '
                        'populated. There are no indications of technical '
                        'errors, exceptions, or empty/invalid fields. The tool '
                        "call succeeded. {'failed_tools': '', 'success': True}",
 'tool_success_result': 'fail',
 'tool_success_sample_input': '[{"role": "user", "content": '
                              '"{\\"tool_calls\\": \\"[TOOL_CALL] '
                              'fetch_weather(location=\\\\\\"New '
                              'York\\\\\\")\\\\n[TOOL_RESULT] '
                              "{'temperature': '22\\\\u00b0C', 'condition': "
                              '\'sunny\', \'humidity\': \'45%\'}\

#### Multiple Successful Tool Calls

In [6]:
# Multiple successful tool executions
multiple_successful_response = [
    {
        "createdAt": "2025-03-26T17:27:35Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_CUdbkBfvVBla2YP3p24uhElJ",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:37Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "tool_call_id": "call_CUdbkBfvVBla2YP3p24uhElJ",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "14°C", "condition": "rainy", "precipitation": "light rain"}}],
    },
    {
        "createdAt": "2025-03-26T17:27:38Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_iq9RuPxqzykebvACgX8pqRW2",
                "name": "send_email",
                "arguments": {
                    "recipient": "your_email@example.com",
                    "subject": "Weather Information for Seattle",
                    "body": "Current weather: 14°C and rainy with light rain.",
                },
            }
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:41Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "tool_call_id": "call_iq9RuPxqzykebvACgX8pqRW2",
        "role": "tool",
        "content": [
            {"type": "tool_result", "tool_result": {"message": "Email successfully sent to your_email@example.com.", "status": "delivered"}}
        ],
    },
    {
        "createdAt": "2025-03-26T17:27:42Z",
        "run_id": "run_zblZyGCNyx6aOYTadmaqM4QN",
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I have successfully sent you an email with the weather information for Seattle.",
            }
        ],
    },
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    },
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    },
]

result = tool_call_success(response=multiple_successful_response, tool_definitions=tool_definitions)
pprint(result)

{'tool_success': 1.0,
 'tool_success_completion_tokens': 70,
 'tool_success_finish_reason': 'stop',
 'tool_success_model': 'gpt-4.1-2025-04-14',
 'tool_success_prompt_tokens': 2488,
 'tool_success_reason': 'Both tool results returned valid, non-empty responses '
                        'with no indication of technical errors, exceptions, '
                        'or failures. The fetch_weather result contains '
                        'weather data, and send_email confirms successful '
                        "delivery. {'failed_tools': '', 'success': True}",
 'tool_success_result': 'pass',
 'tool_success_sample_input': '[{"role": "user", "content": '
                              '"{\\"tool_calls\\": \\"[TOOL_CALL] '
                              'fetch_weather(location=\\\\\\"Seattle\\\\\\")\\\\n[TOOL_RESULT] '
                              "{'temperature': '14\\\\u00b0C', 'condition': "
                              "'rainy', 'precipitation': 'light "
                              

#### Tool Call Success without Tool Definitions

In [7]:
# Successful execution without providing tool definitions
simple_successful_response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_calendar_123",
                "name": "get_calendar",
                "arguments": {"start_date": "2025-01-01", "end_date": "2025-01-31"},
            }
        ],
    },
    {
        "tool_call_id": "call_calendar_123",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"events": [{"date": "2025-01-15", "title": "Team Meeting"}, {"date": "2025-01-20", "title": "Project Review"}]}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I found 2 events in your calendar for January: a Team Meeting on the 15th and a Project Review on the 20th.",
            }
        ],
    }
]

# Evaluation without tool definitions
result = tool_call_success(response=simple_successful_response)
pprint(result)

Failed to filter tool definitions, returning original list. Error: 'NoneType' object is not iterable
Tool definitions could not be parsed, falling back to original definitions: None
Tool definitions could not be parsed, falling back to original definitions: None


{'tool_success': 1.0,
 'tool_success_completion_tokens': 58,
 'tool_success_finish_reason': 'stop',
 'tool_success_model': 'gpt-4.1-2025-04-14',
 'tool_success_prompt_tokens': 2412,
 'tool_success_reason': 'The tool get_calendar returned a result containing a '
                        'list of events without any indication of error, '
                        "exception, or technical failure. {'failed_tools': '', "
                        "'success': True}",
 'tool_success_result': 'pass',
 'tool_success_sample_input': '[{"role": "user", "content": '
                              '"{\\"tool_calls\\": \\"[TOOL_CALL] '
                              'get_calendar(start_date=\\\\\\"2025-01-01\\\\\\", '
                              'end_date=\\\\\\"2025-01-31\\\\\\")\\\\n[TOOL_RESULT] '
                              "{'events': [{'date': '2025-01-15', 'title': "
                              "'Team Meeting'}, {'date': '2025-01-20', "
                              '\'title\': \'Project Revie

#### Example of Failed Tool Call

In [8]:
# Failed tool execution with error
failed_response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_456",
                "name": "fetch_weather",
                "arguments": {"location": "InvalidCity"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_456",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"error": "Location not found", "status": "failed", "code": 404}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I'm sorry, I couldn't retrieve the weather information due to an error.",
            }
        ],
    }
]

tool_definitions = [
    {
        "name": "fetch_weather",
        "description": "Fetches the weather information for the specified location.",
        "parameters": {
            "type": "object",
            "properties": {"location": {"type": "string", "description": "The location to fetch weather for."}},
        },
    }
]

# This should score FALSE due to tool failure
result = tool_call_success(response=failed_response, tool_definitions=tool_definitions)
pprint(result)

{'tool_success': 0.0,
 'tool_success_completion_tokens': 79,
 'tool_success_finish_reason': 'stop',
 'tool_success_model': 'gpt-4.1-2025-04-14',
 'tool_success_prompt_tokens': 2398,
 'tool_success_reason': "The fetch_weather tool result contains an 'error' "
                        "field with 'Location not found', a 'status' of "
                        "'failed', and a 404 code. This clearly indicates a "
                        "technical failure in the tool call. {'failed_tools': "
                        "'fetch_weather', 'success': False}",
 'tool_success_result': 'fail',
 'tool_success_sample_input': '[{"role": "user", "content": '
                              '"{\\"tool_calls\\": \\"[TOOL_CALL] '
                              'fetch_weather(location=\\\\\\"InvalidCity\\\\\\")\\\\n[TOOL_RESULT] '
                              "{'error': 'Location not found', 'status': "
                              '\'failed\', \'code\': 404}\\", '
                              '\\"tool_defini

#### Example of Tool Call Timeout

In [9]:
# Tool execution timeout
timeout_response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_email_789",
                "name": "send_email",
                "arguments": {
                    "recipient": "test@example.com",
                    "subject": "Test Email",
                    "body": "This is a test email."
                },
            }
        ],
    },
    {
        "tool_call_id": "call_email_789",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"error": "Request timeout", "status": "timeout", "timeout_duration": "30s"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I'm unable to send the email due to a timeout error. Please try again later.",
            }
        ],
    }
]

tool_definitions = [
    {
        "name": "send_email",
        "description": "Sends an email with the specified subject and body to the recipient.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {"type": "string", "description": "Email address of the recipient."},
                "subject": {"type": "string", "description": "Subject of the email."},
                "body": {"type": "string", "description": "Body content of the email."},
            },
        },
    }
]

# This should score FALSE due to timeout
result = tool_call_success(response=timeout_response, tool_definitions=tool_definitions)
pprint(result)

{'tool_success': 0.0,
 'tool_success_completion_tokens': 80,
 'tool_success_finish_reason': 'stop',
 'tool_success_model': 'gpt-4.1-2025-04-14',
 'tool_success_prompt_tokens': 2422,
 'tool_success_reason': "The result for send_email contains an 'error' field "
                        "with 'Request timeout' and a 'status' of 'timeout', "
                        'which indicates a technical failure due to timeout. '
                        'Therefore, the send_email tool call failed. '
                        "{'failed_tools': 'send_email', 'success': False}",
 'tool_success_result': 'fail',
 'tool_success_sample_input': '[{"role": "user", "content": '
                              '"{\\"tool_calls\\": \\"[TOOL_CALL] '
                              'send_email(recipient=\\\\\\"test@example.com\\\\\\", '
                              'subject=\\\\\\"Test Email\\\\\\", '
                              'body=\\\\\\"This is a test '
                              'email.\\\\\\")\\\\n[TOOL_RES

#### Mixed Results - Some Success, Some Failure

In [10]:
# Mixed results - one success, one failure
mixed_response = [
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_weather_success",
                "name": "fetch_weather",
                "arguments": {"location": "Seattle"},
            }
        ],
    },
    {
        "tool_call_id": "call_weather_success",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"temperature": "15°C", "condition": "sunny"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_email_fail",
                "name": "send_email",
                "arguments": {
                    "recipient": "invalid-email",
                    "subject": "Weather Report",
                    "body": "Weather is sunny in Seattle."
                },
            }
        ],
    },
    {
        "tool_call_id": "call_email_fail",
        "role": "tool",
        "content": [{"type": "tool_result", "tool_result": {"error": "Invalid email format", "status": "failed"}}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "I got the weather information but failed to send the email due to an invalid email address.",
            }
        ],
    }
]

# This should score FALSE because at least one tool call failed
result = tool_call_success(response=mixed_response, tool_definitions=tool_definitions)
pprint(result)

{'tool_success': 0.0,
 'tool_success_completion_tokens': 71,
 'tool_success_finish_reason': 'stop',
 'tool_success_model': 'gpt-4.1-2025-04-14',
 'tool_success_prompt_tokens': 2444,
 'tool_success_reason': 'The fetch_weather tool returned a valid weather '
                        'result, indicating success. The send_email tool '
                        "returned an error message and status 'failed', which "
                        'is a technical failure. Therefore, the evaluation '
                        "process has failed. {'failed_tools': 'send_email'}",
 'tool_success_result': 'fail',
 'tool_success_sample_input': '[{"role": "user", "content": '
                              '"{\\"tool_calls\\": \\"[TOOL_CALL] '
                              'fetch_weather(location=\\\\\\"Seattle\\\\\\")\\\\n[TOOL_RESULT] '
                              "{'temperature': '15\\\\u00b0C', 'condition': "
                              "'sunny'}\\\\n[TOOL_CALL] "
                              'send_em